In [6]:
sc

In [1]:
lines = sc.textFile("input/file1")
lines

input/file1 MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [7]:
lines.collect()

[u'Hello World Bye World']

In [8]:
words = lines.flatMap(lambda l: l.split(" "))
words.count()

4

## Problem 1

In [14]:
lines = sc.textFile('input')
record = lines.flatMap(lambda l: l.split(" "))\
              .map(lambda a: (a, 1))\
              .reduceByKey(lambda x,y:x+y)\
              .map(lambda x:(x[0],x[1]))\
              .sortByKey(True)
ans = record.take(record.count())
for a in ans:print a[0] + ' : ' + str(a[1])

Bye : 1
Goodbye : 1
Hadoop : 2
Hello : 2
World : 2


## Problem 2

In [15]:
from pyspark.sql.types import *

sqlContext = SQLContext(sc)

lines = sc.textFile("hive/purchases.txt")
record = lines.map(lambda l: l.split(","))

purchases = record.map(lambda p: (p[0].split(" ")[0], p[1], p[2], float(p[3]), p[4]))

fields = [StructField("timestamp", StringType(), True), \
          StructField("location", StringType(), True), \
          StructField("category", StringType(), True), \
          StructField("price", FloatType(), True), \
          StructField("card", StringType(), True), \
         ]

purchasesSchema = StructType(fields)
purchasesDataFrame = sqlContext.createDataFrame(purchases, purchasesSchema)
purchasesTable = purchasesDataFrame.registerTempTable("purchases")

query = "select avg(price) as ans from purchases where card = \"MasterCard\" "
result = sqlContext.sql(query)
ans = result.map(lambda a: str(a.ans))
print "1. What is the average price of the products that were purchased via Mastercard?"
print float(ans.collect()[0])

query = "select timestamp as DATE, sum(price) as SALE from purchases group by timestamp order by SALE DESC LIMIT 1"
result = sqlContext.sql(query)
ans = result.map(lambda a: str(a.DATE))
print "2. Which date recorded the highest total sales?"
print ans.collect()[0]

query = "select min(price) as ans from purchases where category = \"Computers\" "
result = sqlContext.sql(query)
ans = result.map(lambda a: a.ans)
print "3. What is the minimum value of a product under the Computers category?"
print round(float(ans.collect()[0]),2)

query = "select count(*) as ans from (select DISTINCT category from purchases) temp "
result = sqlContext.sql(query)
ans = result.map(lambda a: str(a.ans))
print "4. How many distinct categories of products are there?" 
print int(ans.collect()[0])

query = "select location as ans, sum(price) as SALE from purchases group by location order by SALE ASC LIMIT 1 "
result = sqlContext.sql(query)
ans = result.map(lambda a: str(a.ans))
print "5. Which store location had the lowest total sales?"
print ans.collect()[0]

1. What is the average price of the products that were purchased via Mastercard?
275.067731742
2. Which date recorded the highest total sales?
2012-03-17
3. What is the minimum value of a product under the Computers category?
0.38
4. How many distinct categories of products are there?
18
5. Which store location had the lowest total sales?
Plano
